In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy.stats import ttest_ind
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import scale
from statsmodels.tools.tools import add_constant

%matplotlib inline
tracts_data_path = 'data/features.csv'

C:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
dtypes = {'GEOID' : str,
          'place_geoid' : str,
          'state' : str,
          'county' : str}

df = pd.read_csv(tracts_data_path, encoding='utf-8', dtype=dtypes)
df = df.rename(columns={'GEOID' : 'tract'}).set_index('tract')
assert df.index.is_unique

In [3]:
print(len(df))
df.head()

12328


,land_area,place_geoid,place_name,total_pop,median_age,prop_hispanic,prop_white,prop_black,prop_asian,prop_single_fam_detached,...,"Salt Lake City, UT","San Antonio, TX","San Diego, CA","San Francisco, CA","San Jose, CA","Seattle, WA","St. Louis, MO","Tampa, FL","Virginia Beach, VA","Washington, DC"
tract,,,,,,,,,,,,,,,,,,,,,
01073000100,7549578,0107000,"Birmingham, AL",2970.0,32.2,0.046,0.171,0.785,0.000,0.705,...,0,0,0,0,0,0,0,0,0,0
01073000300,2093104,0107000,"Birmingham, AL",2494.0,36.5,0.180,0.046,0.672,0.084,0.326,...,0,0,0,0,0,0,0,0,0,0
01073000400,8001582,0107000,"Birmingham, AL",3437.0,30.6,0.007,0.079,0.908,0.000,0.897,...,0,0,0,0,0,0,0,0,0,0
01073000500,4819145,0107000,"Birmingham, AL",3735.0,35.8,0.014,0.050,0.929,0.000,0.546,...,0,0,0,0,0,0,0,0,0,0
01073000700,3520564,0107000,"Birmingham, AL",2562.0,25.4,0.025,0.000,0.977,0.000,0.569,...,0,0,0,0,0,0,0,0,0,0


In [4]:
str(sorted(df.columns.tolist()))

"['Atlanta, GA', 'Austin, TX', 'Baltimore, MD', 'Birmingham, AL', 'Boston, MA', 'Buffalo, NY', 'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH', 'Cleveland, OH', 'Columbus, OH', 'Dallas, TX', 'Denver, CO', 'Detroit, MI', 'Hartford, CT', 'Houston, TX', 'Indianapolis, IN', 'Jacksonville, FL', 'Kansas City, MO', 'Las Vegas, NV', 'Los Angeles, CA', 'Louisville, KY', 'Memphis, TN', 'Miami, FL', 'Milwaukee, WI', 'Minneapolis, MN', 'Nashville, TN', 'New Orleans, LA', 'New York, NY', 'Oklahoma City, OK', 'Orlando, FL', 'Philadelphia, PA', 'Phoenix, AZ', 'Pittsburgh, PA', 'Portland, OR', 'Providence, RI', 'Raleigh, NC', 'Richmond, VA', 'Riverside, CA', 'Sacramento, CA', 'Salt Lake City, UT', 'San Antonio, TX', 'San Diego, CA', 'San Francisco, CA', 'San Jose, CA', 'Seattle, WA', 'St. Louis, MO', 'Tampa, FL', 'Virginia Beach, VA', 'Washington, DC', 'bias_bc', 'bias_diff', 'bias_log', 'bias_ratio', 'centroid', 'clist_count', 'count_rental_units', 'count_renter_occupied_units', 'county', 'distance_

## Organize our variables

In [5]:
# get a set of all predictor variables by eliminating the ones that aren't
not_predictors = ['bias_diff', 'bias_ratio', 'bias_log', 'bias_bc', 'centroid', 
                  'clist_count', 'county', 'geometry', 'is_over', 'land_area',
                  'lat_city_center', 'lng_city_center', 'place_geoid', 'place_name', 
                  'proportionate_count', 'state', 'prop_white_change_2012_2015']

predictors_all = df.drop(columns=not_predictors).columns

In [6]:
# get all the predictors that are not dummy variables or log-transformed versions of others
predictors_no_dummies = sorted([p for p in predictors_all if 'dummy' not in p   #race dummies
                                                          and ',' not in p      #city dummies
                                                          and '_log' not in p]) #log-transformed

In [7]:
# get a small subset of key variables of interest
predictors_key = ['med_income_k', 'median_gross_rent_k', 'prop_20_34', 'prop_bachelors_or_higher', 
                  'prop_below_poverty', 'prop_white', 'renter_household_size', 'prop_english_only',
                  'prop_college_grad_student', 'prop_nonrelatives_household']

In [8]:
# create interaction terms
df['white:income'] = df['dummy_white_major'] * df['med_income_k_log']
df['white:education'] = df['dummy_white_major'] * df['prop_bachelors_or_higher']

In [9]:
# the independent variables to include in our regression model (plus the city dummies below)
predictors_model = ['rental_vacancy_rate',
                    'prop_same_residence_year_ago',
                    'distance_to_center_km_log',
                    'mean_travel_time_work_log',
                    'prop_built_before_1940',
                    'med_rooms_in_house',
                    'median_gross_rent_k',
                    'med_income_k_log',
                    'prop_20_34',
                    'prop_college_grad_student',
                    'prop_english_only',
                    'renter_household_size_log',
                    'prop_bachelors_or_higher',
                    'dummy_white_major',
                    'dummy_black_major',
                    'dummy_hispanic_major',
                    'white:income']

In [10]:
# the city dummies to control for regional differences (leave out DC to avoid collinearity)
city_dummies = ['Atlanta, GA', 'Austin, TX', 'Baltimore, MD', 'Birmingham, AL', 'Boston, MA', 
                'Buffalo, NY', 'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH', 'Cleveland, OH', 
                'Columbus, OH', 'Dallas, TX', 'Denver, CO', 'Detroit, MI', 'Hartford, CT', 'Houston, TX', 
                'Indianapolis, IN', 'Jacksonville, FL', 'Kansas City, MO', 'Las Vegas, NV', 
                'Los Angeles, CA', 'Louisville, KY', 'Memphis, TN', 'Miami, FL', 'Milwaukee, WI', 
                'Minneapolis, MN', 'Nashville, TN', 'New Orleans, LA', 'New York, NY', 
                'Oklahoma City, OK', 'Orlando, FL', 'Philadelphia, PA', 'Phoenix, AZ', 
                'Pittsburgh, PA', 'Portland, OR', 'Providence, RI', 'Raleigh, NC', 'Richmond, VA', 
                'Riverside, CA', 'Sacramento, CA', 'Salt Lake City, UT', 'San Antonio, TX', 
                'San Diego, CA', 'San Francisco, CA', 'San Jose, CA', 'Seattle, WA', 'St. Louis, MO', 
                'Tampa, FL', 'Washington, DC']

## Race

In [11]:
len(df)

12328

In [12]:
# in how many tracts is each race the majority?
cols = ['dummy_white_major', 'dummy_black_major', 'dummy_hispanic_major', 'dummy_asian_major']
df[cols].sum()

dummy_white_major       4868
dummy_black_major       2767
dummy_hispanic_major    2266
dummy_asian_major        231
dtype: int64

In [13]:
# in how many tracts is each race the plurality?
cols = ['dummy_white_plural', 'dummy_black_plural', 'dummy_hispanic_plural', 'dummy_asian_plural']
df[cols].sum()

dummy_white_plural       5819
dummy_black_plural       3140
dummy_hispanic_plural    2898
dummy_asian_plural        465
dtype: int64

In [14]:
# what proportion of tracts are over-represented?
round(df['is_over'].sum() / len(df), 4)

0.2508

In [15]:
# what proportion of tracts with each of these races as the majority is over-represented?
white_tracts = df[df['dummy_white_major']==1]
white_odds = round(white_tracts['is_over'].sum() / len(white_tracts), 4)

asian_tracts = df[df['dummy_asian_major']==1]
asian_odds = round(asian_tracts['is_over'].sum() / len(asian_tracts), 4)

black_tracts = df[df['dummy_black_major']==1]
black_odds = round(black_tracts['is_over'].sum() / len(black_tracts), 4)

hisp_tracts = df[df['dummy_hispanic_major']==1]
hisp_odds = round(hisp_tracts['is_over'].sum() / len(hisp_tracts), 4)

print(white_odds, asian_odds, black_odds, hisp_odds)
print(round(white_odds / asian_odds, 4))
print(round(white_odds / black_odds, 4))
print(round(white_odds / hisp_odds, 4))

0.3708 0.2165 0.1662 0.1108
1.7127
2.231
3.3466


Majority white tracts are overrepresented on Craigslist 2x as often as majority black tracts and 3x  as often as majority hispanic tracts.

In [16]:
# what pct of tracts have less than 25% the listings we'd expect proportionally?
df['very_under_rep'] = df['bias_ratio'] < 0.25
df['very_under_rep'].sum() / len(df)

0.238724853990915

In [17]:
# what proportion of tracts with each of these races as the majority is very under-represented?
white_tracts = df[df['dummy_white_major']==1]
white_odds = round(white_tracts['very_under_rep'].sum() / len(white_tracts), 4)

asian_tracts = df[df['dummy_asian_major']==1]
asian_odds = round(asian_tracts['very_under_rep'].sum() / len(asian_tracts), 4)

black_tracts = df[df['dummy_black_major']==1]
black_odds = round(black_tracts['very_under_rep'].sum() / len(black_tracts), 4)

hisp_tracts = df[df['dummy_hispanic_major']==1]
hisp_odds = round(hisp_tracts['very_under_rep'].sum() / len(hisp_tracts), 4)

print(white_odds, asian_odds, black_odds, hisp_odds)

0.1171 0.2424 0.2703 0.4779


## Gini

In [18]:
# gini coefficient measures how evenly some value is distributed among a set of buckets
# we can measure how evenly listings are distributed among tracts
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return round((fair_area - area) / fair_area, 4)

In [19]:
# nationwide
print(gini(df['clist_count']))
print(gini(df['proportionate_count']))
print(gini(df['count_renter_occupied_units']))

0.7958
0.5407
0.3838


The proportionate_count is a function of per-city count_renter_occupied_units, but their gini coefficients don't match nationwide because proportionate_count is assigned as per-city proportions, not nationwide. See below: they match.

In [20]:
# now examine gini coefficients for each city
data = {}
for name, group in df.groupby('place_name'):
    
    data[name] = {'clist_gini' : gini(group['clist_count']),
                  'prop_gini' : gini(group['proportionate_count']),
                  'count_units' : gini(group['count_renter_occupied_units'])}
    
ginis = pd.DataFrame(data).T
ginis['ratio'] = ginis['clist_gini'] / ginis['prop_gini']
ginis.sort_values(by='ratio', ascending=False).round(3)

,clist_gini,count_units,prop_gini,ratio
"Hartford, CT",0.826,0.262,0.258,3.199
"Miami, FL",0.789,0.284,0.282,2.804
"Philadelphia, PA",0.756,0.274,0.273,2.774
"Boston, MA",0.721,0.266,0.267,2.700
"Providence, RI",0.591,0.229,0.223,2.644
"Milwaukee, WI",0.769,0.290,0.292,2.634
"Buffalo, NY",0.616,0.236,0.235,2.616
"Cleveland, OH",0.727,0.290,0.283,2.571
"St. Louis, MO",0.742,0.297,0.294,2.522
"Baltimore, MD",0.773,0.318,0.307,2.517


Higher gini coefficient for actual craigslist listings suggests they are more concentrated into fewer tracts than a proportional distribution would be.

## *t*-tests and effect sizes for significant differences in variables

Divide the data into two subsets: overrepresented and underrepresented, then test if variables' means differ significantly between them.

In [21]:
def significance_05(p):
    if p <= 0.05:
        return '*'
    else:
        return '~~'
    
def significance_05_01_001(p):
    if p <= 0.001:
        return '***'
    if p <= 0.01:
        return '**~~'
    elif p <= 0.05:
        return '*~~~~'
    else:
        return '~~~~~~'

In [22]:
# effect size: as cohen's d
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    d = (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)
    return d

def test_differences(subset1, subset2, variables):
    test_results = {}
    for var in variables:
        a = subset1[var]
        b = subset2[var]
        t_statistic, p_value = ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
        diff = subset1[var].mean() - subset2[var].mean()
        d_value = cohen_d(x=a, y=b)
        test_results[var] = {'diff_mean' : '{:.3f}{}'.format(diff, significance_05(p_value)),
                             't_stat' : round(t_statistic, 2),
                             'p_val' : round(p_value, 3),
                             'cohen_d' : round(d_value, 2)}
    
    return test_results

In [23]:
# divide dataset into overrepresented tracks and not overrepresented
over = df[df['is_over']==1]
under = df[df['is_over']==0]

In [24]:
# variables' effect sizes between over and underrepresented tracts
results = test_differences(over, under, predictors_no_dummies)
effect_sizes = pd.DataFrame(results).T.sort_values('cohen_d', ascending=False)
effect_sizes = effect_sizes.reindex(columns=['cohen_d', 'diff_mean'])
effect_sizes

,cohen_d,diff_mean
prop_bachelors_or_higher,0.81,0.172*
listings_per_unit,0.67,0.038*
med_income_k,0.6,17.356*
prop_white,0.59,0.173*
prop_college_grad_student,0.52,0.097*
median_gross_rent_k,0.5,0.180*
prop_20_34,0.39,0.039*
prop_english_only,0.37,0.094*
prop_nonrelatives_household,0.35,0.021*
med_home_value_k,0.33,74.406*


In [25]:
#effect_sizes['cohen_d'] = effect_sizes['cohen_d'].map(lambda x: '{:.2f}'.format(x))
#print(effect_sizes.to_latex())

"Cohen suggested that d=0.2 be considered a 'small' effect size, 0.5 represents a 'medium' effect size and 0.8 a 'large' effect size. This means that if two groups' means don't differ by 0.2 standard deviations or more, the difference is trivial, even if it is statistically signficant."

Perhaps we can interpret small-medium effect size as absolute value 0.3 <= x < 0.5?

d is not affected by units/sizes. So income and income_k will have same d.

In [26]:
# look at some smaller subset of key variables of interest, per city
city_results = {}
for city, group in df.groupby('place_name'):
    group_over = group[group['is_over']==1]
    group_under = group[group['is_over']==0]
    group_results = test_differences(group_over, group_under, predictors_key)
    var_d = {k:'{:.2f}{}'.format(v['cohen_d'], significance_05(v['p_val'])) for k, v in group_results.items()}
    city_results[city] = var_d

In [27]:
city_effect_sizes = pd.DataFrame(city_results).T
city_effect_sizes.index = city_effect_sizes.index.map(lambda x: x.split(', ')[0])
city_effect_sizes.head(10)
city_effect_sizes.sort_values(by='prop_english_only', ascending=False).head(10)

,med_income_k,median_gross_rent_k,prop_20_34,prop_bachelors_or_higher,prop_below_poverty,prop_college_grad_student,prop_english_only,prop_nonrelatives_household,prop_white,renter_household_size
Providence,0.61~~,1.07*,0.12~~,0.82~~,-0.12~~,0.63~~,0.91*,0.42~~,0.90*,-0.27~~
Los Angeles,0.97*,0.96*,0.04~~,1.11*,-0.58*,0.57*,0.86*,0.07~~,1.02*,-0.69*
San Antonio,0.89*,0.58*,-0.17~~,0.99*,-0.60*,0.34*,0.85*,0.03~~,1.01*,-0.47*
Minneapolis,0.76*,0.71*,0.23~~,1.04*,-0.45*,0.42~~,0.81*,0.16~~,0.90*,-0.57*
Boston,0.41*,1.15*,1.31*,1.39*,0.00~~,1.56*,0.77*,0.93*,1.02*,-1.28*
Phoenix,0.62*,0.47*,-0.12~~,0.59*,-0.58*,0.45*,0.71*,-0.16~~,0.67*,-0.28*
Portland,0.60*,0.68*,0.19~~,0.80*,-0.49*,0.38~~,0.70*,-0.18~~,0.61*,-0.57*
Hartford,0.42~~,0.01~~,0.53~~,1.68*,-0.42~~,1.30~~,0.64~~,0.29~~,1.35*,-1.54*
Dallas,0.87*,0.89*,0.39*,1.06*,-0.70*,0.67*,0.60*,0.41*,0.84*,-0.77*
San Francisco,0.54*,0.39*,0.23~~,0.70*,-0.10~~,0.42*,0.59*,-0.15~~,0.54*,-0.73*


In [28]:
#print(city_effect_sizes.to_latex())

## Estimate regression models to predict Craigslist over- or under-representation

In [29]:
# create design matrix containing predictors (drop nulls), and a response variable vector
X = df[predictors_model + city_dummies].dropna()
y = df.loc[X.index]['bias_log']

In [30]:
# estimate a model across the full data set (all cities)
Xc = add_constant(X)
model = sm.OLS(y, Xc)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               bias_log   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     81.21
Date:                Sat, 24 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:26:14   Log-Likelihood:                -15709.
No. Observations:               12224   AIC:                         3.155e+04
Df Residuals:                   12157   BIC:                         3.205e+04
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [31]:
#print(result.summary().as_latex())

In [32]:
# single column version of results    
results = pd.DataFrame({'params' : result.params,
           'se' : result.bse,
           'p' : result.pvalues})

results['se'] = results['se'].map(lambda x: '{:.3f}'.format(x))
results['sig'] = results['p'].map(lambda p: significance_05_01_001(p))
results['params'] = results['params'].map(lambda x: '{:.3f}'.format(x))
results['params'] = results.apply(lambda row: '{}{}'.format(row['params'], row['sig']), axis=1)
results.index = results.index.map(lambda x: x.split(',')[0].replace('Q(\'', ''))
results = results.reindex(columns=['params', 'se'])
results.head()
#print(results.to_latex())

,params,se
const,-2.584***,0.245
rental_vacancy_rate,0.386**~~,0.138
prop_same_residence_year_ago,-0.512***,0.121
distance_to_center_km_log,-0.134***,0.016
mean_travel_time_work_log,-0.067~~~~~~,0.058


^^ if we get warnings about multicollinearity, but have good VIF scores and significant variables, then check a standardized regression (below) to see if it's just scaling or the intercept/constant causing it (intercept shouldn't cause high condition number if we center/standardize our predictors). A high condition number indicates multicollinearity. Rule of thumb, you want this to be below ~20.

durbin-watson tests for autocorrelation. a value around 1.5 to 2.5 is considered fine.

omnibus tests for normality of residuals; if prob < 0.05, we reject the null hypothesis that they are normally distributed. skew and kurtosis describe their distribution.

jarque-bera tests for normality of residuals; if prob < 0.05, we reject the null hypothesis that they are normally distributed

Interaction term shows that the positive effect of income matters less as tract gets whiter and that the positive effect of white matters less as tract gets richer.

In [33]:
# estimate a standardized model across the full data set (all cities)
y_stdrd = pd.Series(data=zscore(y), index=y.index, name=y.name)
X_stdrd = pd.DataFrame(data=zscore(X), index=X.index, columns=X.columns)
Xc_stdrd = add_constant(X_stdrd)
model_stdrd = sm.OLS(y_stdrd, Xc_stdrd)
result_stdrd = model_stdrd.fit()
print(result_stdrd.summary())

                            OLS Regression Results                            
Dep. Variable:               bias_log   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     81.21
Date:                Sat, 24 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:26:14   Log-Likelihood:                -15113.
No. Observations:               12224   AIC:                         3.036e+04
Df Residuals:                   12157   BIC:                         3.086e+04
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [34]:
#print(result_stdrd.summary().as_latex())

## Regression Diagnostics

In [35]:
# condition number to test for multicollinearity
# rule of thumb, you want this below 20
np.linalg.cond(model_stdrd.exog)

33.54500584087998

In [36]:
# plot observed (y-axis) vs fitted (x-axis)
observed = model.endog #actual response var
fitted = result.fittedvalues #predicted response var

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x=fitted, y=observed, s=0.2)
ax.set_xlabel('fitted')
ax.set_ylabel('observed')
ax.set_title('actual vs predicted')

# draw a 45° y=x line
ax.set_xlim((min(np.append(observed, fitted)), max(np.append(observed, fitted))))
ax.set_ylim((min(np.append(observed, fitted)), max(np.append(observed, fitted))))
ax.plot(ax.get_xlim(), ax.get_ylim(), ls='--', c='k', alpha=0.5)

fig.savefig('images/diagnostic_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.close()

In [37]:
# standardized residuals: the internally studentized residuals
resids_stud = result.get_influence().resid_studentized_internal

In [38]:
# residuals plot for heteroskedasticity
# want this to look like a random point pattern with no discernable trend
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x=result.fittedvalues, y=resids_stud, s=0.2)
ax.axhline(y=0, ls='--', c='k', alpha=0.5)
ax.set_title('residuals vs fitted plot')
ax.set_xlabel('fitted values')
ax.set_ylabel('standardized residuals')

fig.savefig('images/diagnostic_residuals_vs_fitted.png', dpi=300, bbox_inches='tight')
plt.close()

In [39]:
# scale-location plot (aka spread-location plot)
# want this to look like a random point pattern with no discernable trend
resids_stud_abs_sqrt = np.sqrt(np.abs(resids_stud))
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x=result.fittedvalues, y=resids_stud_abs_sqrt, s=0.2)
ax.set_title('scale-location plot')
ax.set_xlabel('fitted values')
ax.set_ylabel('square-root absolute standardized residuals ')

fig.savefig('images/diagnostic_scale_location.png', dpi=300, bbox_inches='tight')
plt.close()

In [40]:
# are residuals approximately normally distributed?
# null hypothesis is normal dist, p-value < 0.05 means reject null
# typically want skew and kurtosis to be within -2 to 2
# but with sufficiently large sample size, we'll always reject the null
jb, jb_p, skew, kurtosis = sms.jarque_bera(resids_stud)
print([round(x, 3) for x in [jb, jb_p, skew, kurtosis]])

[1036.654, 0.0, 0.549, 3.911]


In [41]:
# are residuals approximately normally distributed?
# visuals can be more useful than test-statistics
fig, ax = plt.subplots(figsize=(6, 6))
ax = pd.Series(resids_stud).hist(bins=30, ax=ax)
ax.set_title('standardized residuals histogram')
fig.savefig('images/diagnostic_residuals_histogram.png', dpi=300, bbox_inches='tight')
plt.close()

In [42]:
# are residuals approximately normally distributed?
# you want the points to tightly follow the line
# the hist above and qq plot below are ok, not terrible
fig, ax = plt.subplots(figsize=(6, 6))
fig = sm.qqplot(resids_stud, line='45', ax=ax)
ax.set_title('normal probability plot of the standardized residuals')
fig.savefig('images/diagnostic_residuals_qq_plot.png', dpi=300, bbox_inches='tight')
plt.close()

In [43]:
# create figure and axes
n = len(predictors_model)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
axes = [item for sublist in axes for item in sublist]

resids_stud = result.get_influence().resid_studentized_internal

# for each axis and variable, scatterplot the resids
for ax, var in zip(axes, sorted(predictors_model)):
    ax.scatter(x=X[var], y=resids_stud, s=0.2)
    ax.set_xlabel(var)
    ax.set_ylabel('standardized residuals')

# save to disk and show
fig.savefig('images/scatter_resids_vs_predictors.jpg', bbox_inches='tight', dpi=150)
plt.close()

## Regression model for just one city

In [44]:
# subset data for a single city
place_name = 'New York, NY'
df_city = df[df['place_name']==place_name]
print(sum(df_city['bias_ratio']>1), sum(df_city['bias_ratio']<=1))

445 1667


In [45]:
X_city = df_city[predictors_model]
print(len(X_city))
X_city = X_city.dropna()
y_city = df_city.loc[X_city.index]['bias_log']
print(len(X_city))
Xc_city = add_constant(X_city)

2112
2097


In [46]:
# estimate a model for this single city
model_city = sm.OLS(y_city, Xc_city)
result_city = model_city.fit()
print(result_city.summary())

                            OLS Regression Results                            
Dep. Variable:               bias_log   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     77.51
Date:                Sat, 24 Mar 2018   Prob (F-statistic):          6.34e-207
Time:                        16:26:20   Log-Likelihood:                -2147.3
No. Observations:                2097   AIC:                             4331.
Df Residuals:                    2079   BIC:                             4432.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

## Logit models

In [47]:
X_logit = df[predictors_model]
print(len(X_logit))
X_logit = X_logit.dropna()
y_logit = df.loc[X_logit.index]['is_over']
print(len(X_logit))
Xc_logit = add_constant(X_logit)

12328
12224


In [48]:
%%time
# predict whether or not tract is overrepresented on craigslist (yes/no)
#model_logit = sm.Logit(y_logit, Xc_logit)
#result_logit = model_logit.fit()
#print(result_logit.summary())

Wall time: 0 ns


## Dimensionality reduction

### PCA with the reduced set of predictors

In [49]:
X = df[predictors_model].drop(columns=['white:income']).dropna()
X = pd.DataFrame(scale(X.values), columns=X.columns)

In [50]:
# n dimensions
n = 6
pca = PCA(n_components=n)
pca.fit(X=X)

PCA(copy=True, iterated_power='auto', n_components=6, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [51]:
# amount of variance that each component explains
pca.explained_variance_ratio_

array([0.25679932, 0.18991821, 0.14190001, 0.09245353, 0.06204835,
       0.05167907])

In [52]:
# cumulative variance explained
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3))

array([0.257, 0.447, 0.589, 0.681, 0.743, 0.795])

In [53]:
labels = ['PC{}'.format(i+1) for i in range(n)]
pd.DataFrame(pca.components_, columns=X.columns, index=labels).T.sort_values('PC1', ascending=False).round(3)

,PC1,PC2,PC3,PC4,PC5,PC6
prop_bachelors_or_higher,0.417,-0.199,-0.061,-0.077,0.011,0.105
prop_college_grad_student,0.373,0.168,-0.137,-0.080,0.231,-0.093
dummy_white_major,0.340,-0.224,0.061,0.090,-0.248,0.105
prop_20_34,0.282,0.297,-0.236,0.005,0.208,-0.091
med_income_k_log,0.239,-0.443,-0.104,-0.032,0.065,0.115
prop_english_only,0.204,-0.009,0.555,-0.063,-0.006,-0.106
median_gross_rent_k,0.180,-0.353,-0.236,-0.181,0.273,0.051
prop_built_before_1940,0.041,0.148,-0.055,-0.588,-0.413,0.275
rental_vacancy_rate,-0.043,0.165,0.219,0.198,0.314,0.879
med_rooms_in_house,-0.071,-0.384,0.318,0.124,-0.195,-0.004


### Factor analysis with the reduced set of predictors

In [54]:
# n factors
n = 6
fa = FactorAnalysis(n_components=n, max_iter=5000)
fa.fit(X=X)

FactorAnalysis(copy=True, iterated_power=3, max_iter=5000, n_components=6,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

In [55]:
labels = ['Fac{}'.format(i+1) for i in range(n)]
pd.DataFrame(fa.components_, columns=X.columns, index=labels).T.sort_values('Fac1', ascending=False).round(3)

,Fac1,Fac2,Fac3,Fac4,Fac5,Fac6
prop_20_34,0.473,-0.038,-0.306,0.323,0.589,0.113
prop_college_grad_student,0.459,-0.320,-0.361,0.281,0.380,0.149
prop_built_before_1940,0.361,0.099,-0.009,-0.041,-0.130,0.318
prop_english_only,0.244,-0.743,0.553,-0.143,0.093,0.021
prop_bachelors_or_higher,0.238,-0.679,-0.570,-0.092,0.038,0.027
dummy_white_major,0.105,-0.635,-0.298,-0.082,0.009,-0.257
dummy_black_major,0.072,-0.004,0.720,-0.015,-0.038,0.444
rental_vacancy_rate,0.062,0.012,0.270,0.111,0.059,-0.072
dummy_hispanic_major,-0.151,0.668,-0.216,0.052,-0.041,-0.191
median_gross_rent_k,-0.163,-0.288,-0.620,-0.377,0.072,0.311
